# Lesson 02 - CRAM Basics

## Loading the tutorial package

`ros-load:load-system` loads an ASDF (Another System Definition Facility) syste, from within a Catkin package.

In [1]:
(asdf:load-system :cram-pr2-pick-place-demo)

T

BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/cl_store/src/cl-store.asd" contains definition for system "cl-store-tests". Please only define "cl-store" and secondary systems with a name starting with "cl-store/" (e.g. "cl-store/test") in that file.
BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/split_sequence/src/split-sequence.asd" contains definition for system "split-sequence-tests". Please only define "split-sequence" and secondary systems with a name starting with "split-sequence/" (e.g. "split-sequence/test") in that file.
BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/cl_ppcre/src/cl-ppcre.asd" contains definition for system "cl-ppcre-test". Please only define "cl-ppcre" and secondary systems with a name starting with "cl-ppcre/" (e.g. "cl-ppcre/test") in that file.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CRAM-MANIPULATION-INTERFACES::SYMBOL-TO-PROLOG-RULE 

In [91]:
(defpackage cram-bullet-world-tutorial
  (:nicknames #:btw-tut)
   (:use #:common-lisp #:cram-prolog
        #:desig #:exe)
  (:export))

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

In [92]:
(in-package :btw-tut)

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

Now to launch a simulation with a kitchen environment and our robot, type the following command into your Lisp command line:

In [4]:
;; Warning: resets the core processes of the simulation
(roslisp-utilities:startup-ros)

NIL

[(ROSLISP TOP) INFO] 1663443194.466: Node name is /cram_hl_449556_1663443194
[(ROSLISP TOP) INFO] 1663443194.467: Namespace is /
[(ROSLISP TOP) INFO] 1663443194.467: Params are NIL
[(ROSLISP TOP) INFO] 1663443194.467: Remappings are:
[(ROSLISP TOP) INFO] 1663443194.467: master URI is 127.0.0.1:11311
[(ROSLISP TOP) INFO] 1663443197.708: Node startup complete
[(ROSNODE) INFO] 1663443197.713: ROS init #<FUNCTION CRAM-TF::INIT-TF>.
[(CRAM-TF INIT-TF) INFO] 1663443197.713: Set *fixed-frame* to "map".
[(CRAM-TF INIT-TF) INFO] 1663443197.736: Environment name is IAI-KITCHEN.
[(CRAM-TF INIT-TF) INFO] 1663443197.771: Initialized *transformer* to a TRANSFORM-LISTENER.
[(CRAM-TF INIT-TF) INFO] 1663443197.772: *tf-default-timeout* is 4.0.
[(CRAM-TF INIT-TF) INFO] 1663443197.788: Robot name is PR2.
[(CRAM-TF INIT-TF) INFO] 1663443197.793: Set *ODOM-FRAME* to "odom_combined".
[(CRAM-TF INIT-TF) INFO] 1663443197.793: Set *ROBOT-BASE-FRAME* to "base_footprint".
[(CRAM-TF INIT-TF) INFO] 1663443197.793:

In [10]:
;; Warning: resets simulation and deletes objects
(demo::init-projection)

(:BLUE-METAL-PLATE :BOTTLE :BOWL :BREAKFAST-CEREAL :BUTTERMILK :CEREAL :CUBE
 :CUP :FORK :KNIFE :MILK :MILK :RED-METAL-PLATE :SPOON :SPOON :TRAY-BASE :TRAY)

[(SET-ROBOT-STATE-FROM-TF) WARN] 1663438539.241: Failed with transform-stamped-error:
    No transform was published between frames base_footprint and map
    Ignore this warning if no real robot is running.


In [11]:
;; Warning: respawning the window may result in a kernel crash. Restart the lecture via docker to recover from that.
(btr-utils:reset-debug-window)

NIL

## Moving Around



In [93]:
(defun make-pose (reference-frame pose)
    "Creates a cl-transforms pose in 3D space w.r.t. the given `reference-frame' and `pose'.
    The pose is given as a list of two lists, where the first are the x y z coordinates, 
    the second is the quaternion of qx qy qz w, like this: '((2.0 0.0 1.5) (0.0 0.0 0.0 1.0))"
  (cl-transforms-stamped:make-pose-stamped
   reference-frame 0.0
   (apply #'cl-transforms:make-3d-vector (first pose))
   (apply #'cl-transforms:make-quaternion (second pose))))

MAKE-POSE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CRAM-BULLET-WORLD-TUTORIAL::MAKE-POSE in DEFUN


In [94]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal (make-pose "map" '((0 1 0) (0 0 0 1)))))
    (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

#<HASH-TABLE :TEST EQUAL :COUNT 90 {1011FA0AB3}>

In [9]:
(defparameter *base-pose-near-table*
  (make-pose "map" 
             '((-2.2d0 -0.20d0 0.0d0) 
               (0.0d0 0.0d0 -0.707d0 0.707d0))))

*BASE-POSE-NEAR-TABLE*

In [95]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
     (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

#<HASH-TABLE :TEST EQUAL :COUNT 90 {1011038F73}>

Advanced definition of poses with euler angles provides a quaternion.

In [11]:
(cl-transforms:euler->quaternion :az (* pi 0.5)) ;; rotation of pi/2 around the z-axis = 90 degree to the left

#<CL-TRANSFORMS:QUATERNION (0.0d0 0.0d0 0.7071067811865475d0 0.7071067811865476d0)>

The resulting quaternion can be used to specify rotations around an axis. Here we a creating a pose without the help of the `make-pose` function. Fiddle arbound with the `-0.50` factor to PI, and move the robot again, to see the resulting rotation.

In [16]:
(defparameter *base-pose-near-table*
  (cl-transforms-stamped:make-pose-stamped 
   "map" 0.0 
   (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
   (cl-transforms:euler->quaternion :az (* pi -0.50))))

*BASE-POSE-NEAR-TABLE*

## Spawning Objects

In [96]:
(defun spawn-object (spawn-pose &optional (obj-type :bottle) (obj-name 'bottle-1) (obj-color '(1 0 0)))
    "Spawns an object into simulation.
    spawn-pose: list - pose coordinates, like this: '((x y z) (qx qy qz w))
    optional:
      obj-type: keyword - the type of object to spawn, default = :bottle (case-insensitive)
      obj-name: symbol - name of the object, default = 'bottle-1 (case-insensitive)
      obj-color: list - (r g b) color value for the object with range 0.0-1.0, default is red = '(1 0 0)"
  (unless (assoc obj-type btr::*mesh-files*)
    (btr:add-objects-to-mesh-list "cram_pr2_pick_place_demo"))
  (btr-utils:spawn-object obj-name obj-type :color obj-color :pose spawn-pose)
  (btr:simulate btr:*current-bullet-world* 10))

SPAWN-OBJECT

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CRAM-BULLET-WORLD-TUTORIAL::SPAWN-OBJECT in DEFUN


To find all possible `obj-type`, the parameter `btr::*mesh-files*` contains the mesh files as key-value pairs between keyword and the location of their mesh-file. We can map over that list and only return their keys. 

In [19]:
;; the parameter btr::*mesh-files* contains the mesh files 
(map 'list #'first btr::*mesh-files*)

(:TRAY-BASE :SPOON :RED-METAL-PLATE :MILK :CUBE :CEREAL :BUTTERMILK
 :BREAKFAST-CEREAL :BOTTLE :BLUE-METAL-PLATE :MUG :MUG-COMPOUND :PLATE
 :PLATE-COMPOUND :TRAY :TRAY-COMPOUND :CUP-NON-COMPOUND :CUP :MONDAMIN :POT
 :WEISSWURST :BOWL-ORIGINAL :BOWL-NON-COMPOUND :BOWL :FORK :KNIFE :SPATULA :CAP
 :GLASSES :GLOVE :SHOE :ARROW)

Spawn bottle

In [108]:
(spawn-object '((-2.323 -1 0.82) (0 0 0 1)))

NIL

In [21]:
(spawn-object '((1.44 1.28 0.85) (0 0 0 1)) :cube 'cube-1)

NIL

In [22]:
(spawn-object '((0 0 0.1) (0 0 0 1)) :weisswurst 'wurst-1 '(0 1 0))

NIL

In [23]:
(demo::initialize)

NIL

[(SET-ROBOT-STATE-FROM-TF) WARN] 1663443470.097: Failed with transform-stamped-error:
    No transform was published between frames base_footprint and map
    Ignore this warning if no real robot is running.


## Visualizating coordinates

In [27]:
(btr:add-vis-axis-object
 (cl-transforms-stamped:make-pose-stamped
  "map"
  0.0
  (cl-transforms:make-3d-vector 0.43 1.6 1.0126)
  (cl-transforms:make-quaternion 0 1 0 0)))

(#<CL-BULLET:RIGID-BODY {1012D23893}> #<CL-BULLET:RIGID-BODY {1012D24D13}>
 #<CL-BULLET:RIGID-BODY {1012D261A3}>)

In [28]:
(btr:add-vis-axis-object *base-pose-near-table*)

(#<CL-BULLET:RIGID-BODY {100CBFB7F3}> #<CL-BULLET:RIGID-BODY {100CBFCC73}>
 #<CL-BULLET:RIGID-BODY {100CBFE103}>)

blabla

In [29]:
(btr:add-vis-axis-object 'bottle-1)

(#<CL-BULLET:RIGID-BODY {10035D5EB3}> #<CL-BULLET:RIGID-BODY {10035D7333}>
 #<CL-BULLET:RIGID-BODY {10035D87C3}>)

In [30]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-environment-object) 'btr:urdf) 'cl-urdf:links))

("room_link" "world" "sink_area_footprint" "sink_area" "sink_area_surface"
 "sink_area_sink" "sink_area_right_panel" "sink_area_trash_drawer_main"
 "sink_area_trash_drawer_handle" "sink_area_left_upper_drawer_main"
 "sink_area_left_upper_drawer_handle" "sink_area_left_middle_drawer_main"
 "sink_area_left_middle_drawer_handle" "sink_area_left_bottom_drawer_main"
 "sink_area_left_bottom_drawer_handle" "sink_area_dish_washer_main"
 "sink_area_dish_washer_door" "sink_area_dish_washer_door_handle"
 "sink_area_dish_washer_tray_bottom" "sink_area_dish_washer_tray_left_side"
 "sink_area_dish_washer_tray_right_side" "sink_area_dish_washer_tray_back_side"
 "sink_area_dish_washer_tray_front_side"
 "sink_area_dish_washer_tray_handle_left_side"
 "sink_area_dish_washer_tray_handle_right_side"
 "sink_area_dish_washer_tray_handle_front_side" "oven_area_area_footprint"
 "oven_area_area" "oven_area_oven_main" "oven_area_oven_door"
 "oven_area_oven_door_handle" "oven_area_oven_panel"
 "oven_area_oven_kno

In [37]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-environment-object) "sink_area_left_middle_drawer_handle"))

(#<CL-BULLET:RIGID-BODY {1011BCEFB3}> #<CL-BULLET:RIGID-BODY {1011BD0433}>
 #<CL-BULLET:RIGID-BODY {1011BD18C3}>)

In [32]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-robot-object) 'btr:urdf) 'cl-urdf:links))

("base_link" "base_footprint" "base_bellow_link" "base_laser_link"
 "fl_caster_rotation_link" "fl_caster_l_wheel_link" "fl_caster_r_wheel_link"
 "fr_caster_rotation_link" "fr_caster_l_wheel_link" "fr_caster_r_wheel_link"
 "bl_caster_rotation_link" "bl_caster_l_wheel_link" "bl_caster_r_wheel_link"
 "br_caster_rotation_link" "br_caster_l_wheel_link" "br_caster_r_wheel_link"
 "torso_lift_link" "l_torso_lift_side_plate_link"
 "r_torso_lift_side_plate_link" "torso_lift_motor_screw_link" "imu_link"
 "head_pan_link" "head_tilt_link" "head_plate_frame" "sensor_mount_link"
 "high_def_frame" "high_def_optical_frame" "double_stereo_link"
 "wide_stereo_link" "wide_stereo_optical_frame"
 "wide_stereo_l_stereo_camera_frame"
 "wide_stereo_l_stereo_camera_optical_frame"
 "wide_stereo_r_stereo_camera_frame"
 "wide_stereo_r_stereo_camera_optical_frame" "narrow_stereo_link"
 "narrow_stereo_optical_frame" "narrow_stereo_l_stereo_camera_frame"
 "narrow_stereo_l_stereo_camera_optical_frame"
 "narrow_stereo_

In [38]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "l_gripper_tool_frame"))

(#<CL-BULLET:RIGID-BODY {100EFF7403}> #<CL-BULLET:RIGID-BODY {100EFF8883}>
 #<CL-BULLET:RIGID-BODY {100EFF9D13}>)

In [44]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "base_laser_link"))

(#<CL-BULLET:RIGID-BODY {10142AF403}> #<CL-BULLET:RIGID-BODY {10142B0883}>
 #<CL-BULLET:RIGID-BODY {10142B1D13}>)

In [43]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "torso_lift_link"))

(#<CL-BULLET:RIGID-BODY {10114D5F13}> #<CL-BULLET:RIGID-BODY {10114D7393}>
 #<CL-BULLET:RIGID-BODY {10114D8833}>)

In [45]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "wide_stereo_optical_frame"))

(#<CL-BULLET:RIGID-BODY {100DCC1833}> #<CL-BULLET:RIGID-BODY {100DCC2CB3}>
 #<CL-BULLET:RIGID-BODY {100DCC4143}>)

In [46]:
(btr-utils:visualize-designator-costmaps
        (a location
           (reachable-for pr2)
           (arm left)
           (object (an object
                       (type cupboard)
                       (urdf-name sink-area-left-middle-drawer-handle)
                       (part-of iai-kitchen)))))

#<CL-TRANSFORMS-STAMPED:POSE-STAMPED 
   FRAME-ID: "map", STAMP: 0.0
   #<3D-VECTOR (0.2599997818470001d0 0.5399999618530273d0 0.0d0)>
   #<QUATERNION (0.0d0 0.0d0 0.18142028980215855d0 0.98340565304868d0)>>

[BTR-UTILS CM] Visualizing costmaps of #<A LOCATION
    (REACHABLE-FOR PR2)
    (ARM LEFT)
    (OBJECT #<A OBJECT
    (TYPE CUPBOARD)
    (URDF-NAME SINK-AREA-LEFT-MIDDLE-DRAWER-HANDLE)
    (PART-OF IAI-KITCHEN)>)>
[BTR-UTILS CM] Costmap named ENVIRONMENT-FREE-SPACE
[BTR-UTILS CM] Costmap named STATIC-OCCUPIED
[BTR-UTILS CM] Costmap named #<POSE-DISTRIBUTION-RANGE-EXCLUDE-GENERATOR {1005310003}>
[BTR-UTILS CM] Combined costmap


## Intermezzo: About Designators

Action Designators: Going is an Action like searching, looking, fetching, delivering etc. 

Object Designators

Location Designators

Motion Designators

## Perceiving Objects

In [107]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
    (cpl:par
      ;; Moving the robot near the table.
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?navigation-goal)))))
      ;; Increasing the height of the torso by setting the joint angle to 0.3 meters
      (perform (a motion
                  (type moving-torso)
                  (joint-angle 0.3)))
      (perform (an action
                  (type positioning-arm)
                  (left-configuration park)
                  (right-configuration park))))))

#<HASH-TABLE :TEST EQUAL :COUNT 90 {1017F5E213}>

In [99]:
(defparameter *downward-look-coordinate*
 (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1))))
;; This coordinate frame has base_footprint as reference, which is the reference
;; frame of PR2's base.

*DOWNWARD-LOOK-COORDINATE*

In [109]:
(urdf-proj:with-simulated-robot
  (let ((?looking-direction *downward-look-coordinate*))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction)))))))

NIL

In [110]:
(urdf-proj:with-simulated-robot
  (perform (an action
               (type detecting)
               (object (an object
                          (type bottle))))))

#<A OBJECT
    (TYPE BOTTLE)
    (NAME BOTTLE-1)
    (POSE ((:POSE
            #<CL-TRANSFORMS-STAMPED:POSE-STAMPED 
   FRAME-ID: "base_footprint", STAMP: 0.0
   #<3D-VECTOR (0.8088114738033475d0 -0.1186617297137702d0 0.8016609153747559d0)>
   #<QUATERNION (0.003436738297774528d0 0.005772709800403691d0 0.7078478439691029d0 0.7063330366522225d0)>>)
           (:TRANSFORM
            #<CL-TRANSFORMS-STAMPED:TRANSFORM-STAMPED 
   FRAME-ID: "base_footprint", CHILD-FRAME-ID: "bottle_1", STAMP: 0.0
   #<3D-VECTOR (0.8088114738033475d0 -0.1186617297137702d0 0.8016609153747559d0)>
   #<QUATERNION (0.003436738297774528d0 0.005772709800403691d0 0.7078478439691029d0 0.7063330366522225d0)>>)
           (:POSE-IN-MAP
            #<CL-TRANSFORMS-STAMPED:POSE-STAMPED 
   FRAME-ID: "map", STAMP: 0.0
   #<3D-VECTOR (-2.3186617533365887d0 -1.0088114420572916d0 0.8016609191894531d0)>
   #<QUATERNION (0.006512063089758158d0 0.0016517812618985772d0 0.0010711305076256394d0 0.9999768733978271d0)>>)
         

In [102]:
(defparameter *perceived-bottle* nil)
(urdf-proj:with-simulated-robot
    (setf *perceived-bottle* 
          (perform (an action
                       (type detecting)
                       (object (an object
                                   (type bottle)))))))

*PERCEIVED-BOTTLE*

#<A OBJECT
    (TYPE BOTTLE)
    (NAME BOTTLE-1)
    (POSE ((:POSE
            #<CL-TRANSFORMS-STAMPED:POSE-STAMPED 
   FRAME-ID: "base_footprint", STAMP: 0.0
   #<3D-VECTOR (0.8089118639240022d0 -0.11868639808682913d0 0.8016715965270996d0)>
   #<QUATERNION (0.0038287352764987143d0 0.006148431509879053d0 0.7077190849331083d0 0.7064568223600078d0)>>)
           (:TRANSFORM
            #<CL-TRANSFORMS-STAMPED:TRANSFORM-STAMPED 
   FRAME-ID: "base_footprint", CHILD-FRAME-ID: "bottle_1", STAMP: 0.0
   #<3D-VECTOR (0.8089118639240022d0 -0.11868639808682913d0 0.8016715965270996d0)>
   #<QUATERNION (0.0038287352764987143d0 0.006148431509879053d0 0.7077190849331083d0 0.7064568223600078d0)>>)
           (:POSE-IN-MAP
            #<CL-TRANSFORMS-STAMPED:POSE-STAMPED 
   FRAME-ID: "map", STAMP: 0.0
   #<3D-VECTOR (-2.3186864217122394d0 -1.0089118321736654d0 0.8016716003417969d0)>
   #<QUATERNION (0.007054922170937061d0 0.0016402729088440537d0 8.9255440980196d-4 0.9999733567237854d0)>>)
          

## Picking up objects

In [103]:
(urdf-proj:with-simulated-robot 
  (let ((?perceived-bottle *perceived-bottle*))
   (perform (an action
                (type picking-up)
                 (arm (right))
                 (grasp right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (right))))))

NIL

## Placing Objects

In [133]:
(defparameter *final-object-destination*
  (make-pose "map" '((-0.8 2 0.9) (0 0 0 1))))
 
(defparameter *base-pose-near-counter*
  (make-pose "map" '((-0.150d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))

*FINAL-OBJECT-DESTINATION*

*BASE-POSE-NEAR-COUNTER*

In [125]:
(roslisp-utilities:startup-ros)

NIL

[(ROSNODE) INFO] 1663448222.960: ROS cleanup #<FUNCTION CRAM-TF::DESTROY-TF>.
[(ROSNODE) INFO] 1663448222.960: ROS cleanup #<FUNCTION CRAM-TF::DESTROY-TF-BROADCASTER>.
[(ROSNODE) INFO] 1663448222.960: ROS cleanup #<FUNCTION CRAM-TF::DESTROY-MARKER-PUBLISHER>.
[(ROSNODE) INFO] 1663448222.960: ROS cleanup #<FUNCTION CRAM-BULLET-REASONING-BELIEF-STATE::DESTROY-JOINT-STATE-PUB>.
[(ROSLISP TOP) INFO] 1663448226.476: Shutdown complete
[(ROSLISP TOP) INFO] 1663448226.478: Node name is /cram_hl_475914_1663448226
[(ROSLISP TOP) INFO] 1663448226.478: Namespace is /
[(ROSLISP TOP) INFO] 1663448226.478: Params are NIL
[(ROSLISP TOP) INFO] 1663448226.478: Remappings are:
[(ROSLISP TOP) INFO] 1663448226.478: master URI is 127.0.0.1:11311
[(ROSLISP TOP) INFO] 1663448227.486: Node startup complete
[(ROSNODE) INFO] 1663448227.487: ROS init #<FUNCTION CRAM-TF::INIT-TF>.
[(CRAM-TF INIT-TF) INFO] 1663448227.487: Set *fixed-frame* to "map".
[(CRAM-TF INIT-TF) INFO] 1663448227.515: Environment name is IAI-K

In [137]:
(demo::initialize)

(spawn-object '((-2.323 -1 0.82) (0 0 0 1)))

(urdf-proj:with-simulated-robot
 (let ((?navigation-goal 
        (cl-transforms-stamped:make-pose-stamped 
            "map" 0.0 
         (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
         (cl-transforms:euler->quaternion :az (* pi -0.50)))))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 

  (let ((?looking-direction (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1)))))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction))))))
 
 (let ((?perceived-bottle 
        (perform (an action
                     (type detecting)
                     (object (an object
                                 (type bottle))))))
       (?nav-goal (make-pose "map" '((-0.15d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))
       (?drop-pose (make-pose "map" '((-0.8 2 0.9) (0 0 0 1)))))
  
  (perform (an action
                (type picking-up)
                 (arm (right))
                 ;(side right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (perform (an action
                 (type parking-arms)))
   (perform (a motion 
               (type moving-torso)
               (joint-angle 0.3)))
   (perform (an action
                (type going)
                (target (a location
                           (pose ?nav-goal)))))
 
   (perform (an action
                (type placing)
                (arm (right))
                (object ?perceived-bottle)
                (target (a location
                           (pose ?drop-pose)))))))

NIL

NIL

1

[(SET-ROBOT-STATE-FROM-TF) WARN] 1663448620.515: Failed with transform-stamped-error:
    No transform was published between frames base_footprint and map
    Ignore this warning if no real robot is running.


In [112]:


(defun move-kitchen-joint (&key (joint-name "iai_fridge_door_joint")
                             (joint-angle 0.2d0))
  (btr:set-robot-state-from-joints
   `((,joint-name  ,joint-angle))
   (btr:get-environment-object)))

(defun add-objects-to-mesh-list (&optional (ros-package "cram_bullet_world_tutorial"))
  (mapcar (lambda (object-filename-and-object-extension)
            (declare (type list object-filename-and-object-extension))
            (destructuring-bind (object-filename object-extension)
                object-filename-and-object-extension
              (let ((lisp-name (roslisp-utilities:lispify-ros-underscore-name
                                object-filename :keyword)))
                (pushnew (list lisp-name
                               (format nil "package://~a/resource/~a.~a"
                                       ros-package object-filename object-extension)
                               nil)
                         btr::*mesh-files*
                         :key #'car)
                lisp-name)))
          (mapcar (lambda (pathname)
                    (list (pathname-name pathname) (pathname-type pathname)))
                  (directory (physics-utils:parse-uri
                              (format nil "package://~a/resource/*.*" ros-package))))))
                              

;;;; **** ARM Parking
(defun park-arms ()
  (pp-plans::park-arms))

(defun park-arm (arm)
  (pp-plans::park-arms :arm arm))

;;**** Failure Handler
(defmacro handle-failure (errors program-body &body error-handler-body)
  `(progn
    (cpl:with-failure-handling
        ((,errors (e)
           (print e)
           ,@error-handler-body))
    ,@program-body)))


(defparameter *pose-bottle-1*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -2 -0.9d0 0.86d0)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-bottle-2*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.8 2 0.9)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-meal-table*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.15 2.0 0)
   (cl-transforms:make-quaternion 0.0d0 0.0d0 -1.0d0 0.0d0)))

(defparameter *pose-counter*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -1.8547d0 -0.381d0 0.0d0)
   (cl-transforms:axis-angle->quaternion (cl-transforms:make-3d-vector 0 0 1) (/ pi -2))))

(defun spawn-two-bottles ()
  (unless (assoc :bottle btr::*mesh-files*)
    (add-objects-to-mesh-list))
  (prolog:prolog
   `(and (btr:bullet-world ?world)
         (assert (btr:object ?world :mesh bottle-1 ((-2 -0.9 0.860) (0 0 0 1))
                             :mass 0.2 :color (1 0 0) :mesh :bottle))
         (assert (btr:object ?world :mesh bottle-2 ((-0.8 2 0.9) (0 0 0 1))
                             :mass 0.2 :color (0 1 0) :mesh :bottle))
         (btr:simulate ?world 100))))

(defun navigate-to (?navigation-goal)
  (exe:perform (desig:a motion
                        (type going)
                        (pose ?navigation-goal))))

(defun look-at (?point-of-interest)
  (exe:perform (desig:a motion
                        (type looking)
                        (pose ?point-of-interest))))

(defun get-perceived-bottle-desig ()
  (let* ((?bottle-desig (desig:an object (type bottle)))
         (?perceived-bottle-desig (exe:perform
                                   (desig:a motion
                                            (type detecting)
                                            (object ?bottle-desig)))))
    ?perceived-bottle-desig))

(defun pick-up (?object-designator &optional (?arm :right))
  (exe:perform (desig:an action
                         (type picking-up)
                         (arm ?arm)
                         (object ?object-designator))))

(defun place-down (?pose ?object ?arm)
  (exe:perform (desig:an action
                         (type placing)
                         (arm ?arm)
                         (object ?object)
                         (target (desig:a location (pose ?pose))))))

(defun test-switch-two-bottles ()
  (spawn-two-bottles)
  (proj:with-projection-environment urdf-proj:urdf-bullet-projection-environment
    (cpl:top-level
      ;; Go to counter top and perceive bottle
      (let ((?navigation-goal *pose-counter*)
            (?ptu-goal 
              (cl-transforms-stamped:make-pose-stamped
               "base_footprint"
               0.0
               (cl-transforms:make-3d-vector 0.65335d0 0.076d0 0.758d0)
               (cl-transforms:make-identity-rotation))))
        (cpl:par
          ;; Move torso up
          (exe:perform
           (desig:a motion (type moving-torso) (joint-angle 0.3)))
          (exe:perform
           (desig:an action
                     (type parking-arms)))
          (navigate-to ?navigation-goal))
        (look-at ?ptu-goal))
      ;; Pick up bottle-1 with right arm.
      (let ((?perceived-bottle-1 (get-perceived-bottle-desig)))
        (pick-up ?perceived-bottle-1 :right)
        (exe:perform
         (desig:an action
                   (type parking-arms)
                   (arms (:right))))
        ;; Move to the meal table
        (let ((?pose *pose-meal-table*))
          (navigate-to ?pose))
        ;; Pick up bottle-2 with left arm
        (let ((?perceived-bottle-2 (get-perceived-bottle-desig)))
          (pick-up ?perceived-bottle-2 :left)
          ;; Move left arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:left))))
          ;; Place bottle-1 on second table
          (let ((?drop-pose *pose-bottle-2*))
            (place-down ?drop-pose ?perceived-bottle-1 :right))
          ;; Move right arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:right))))
          ;; Move to the counter table
          (let ((?navigation-goal *pose-counter*))
            (navigate-to ?navigation-goal))
          ;; Place bottle-2 on the counter
          (let ((?drop-pose *pose-bottle-1*))
            (place-down ?drop-pose ?perceived-bottle-2 :left))
          (exe:perform
           (desig:an action
                     (type parking-arms))))))))

MOVE-KITCHEN-JOINT

ADD-OBJECTS-TO-MESH-LIST

PARK-ARMS

PARK-ARM

HANDLE-FAILURE

...

LOOK-AT

GET-PERCEIVED-BOTTLE-DESIG

PICK-UP

PLACE-DOWN

TEST-SWITCH-TWO-BOTTLES